In [2]:
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from keras import backend as K


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import tensorflow as tf
import os

Using TensorFlow backend.


In [10]:
data_dir='/home/ubuntu/intelliscope/new_split_basedon_rawpcg/pcg_a'

In [ ]:

#!pip install neptune-client
import os
import neptune
NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNjBlODllY2ItMDcyZC00OGNmLThjZWYtZjM2MjE4Y2M5ZDM0In0="
os.environ['NEPTUNE_PROJECT']="icassp/PCG"
#os.environ['NEPTUNE_NOTEBOOK_ID']="4897a223-8a9f-4f00-8cec-22eda690858d"
os.environ['NEPTUNE_NOTEBOOK_PATH']='test_on_trainA.ipynb'

neptune.init('icassp/PCG',NEPTUNE_API_TOKEN)
exp = neptune.create_experiment(description='Experiment(PCG-48).h5- trainA testing',params=PARAMS,upload_source_files=['test_on_trainA.ipynb'],tags=PARAMS['results_on_trainA'],upload_stdout=True)

In [ ]:
model_dir='/home/ubuntu/intelliscope/notebooks/physionet2016/pcg/models/Experiment(PCG-48).h5'
model=tf.keras.load_model(model_dir)
k1,k2=272, 462
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        data_dir,  # this is the target directory
        target_size=(k1, k2),  # all images will be resized to 150x150
        batch_size=len(os.listdir(data_dir+'/1'))+len(os.listdir(data_dir+'/0')),
        class_mode='categorical')

X,y = test_generator.next()
prediction = model.predict(X)

import numpy as np

y_test=np.argmax(y,axis=1)
y_pred=prediction
from neptunecontrib.monitoring.metrics import *

threshold = 0.5

log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
log_classification_report(y_test, y_pred[:, 1] > threshold)
log_class_metrics(y_test, y_pred[:, 1] > threshold)
log_class_metrics_by_threshold(y_test, y_pred[:, 1])
log_roc_auc(y_test, y_pred)
log_precision_recall_auc(y_test, y_pred)
log_brier_loss(y_test, y_pred[:, 1])
log_log_loss(y_test, y_pred)
log_ks_statistic(y_test, y_pred)
log_cumulative_gain(y_test, y_pred)
log_lift_curve(y_test, y_pred)
log_prediction_distribution(y_test, y_pred[:, 1])
neptune.stop()